In [9]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [5]:
#thread management
from threading import Thread,Lock
from collections import deque

#image manipulation
import numpy as np
import pandas as pd
import cv2 #OpenCV
import matplotlib.pyplot as plt
%matplotlib inline

#general modules
import time
import pickle

#camera modules
#import picamera as pc
#from picamera.array import PiRGBArray

# Streaming Video Analysis in Python 

** Written By:
** Matthew Rubashkin and Colin Higgins

At SVDS we have [*analyzed Caltrain
delays*](http://www.svds.com/the-trains-project-analyzing-caltrain-delays/)
to try to improve Caltrain arrival predictions using real time publicly
available data. However, there were some inconsistencies with the
station arrival time data we pulled from the API. In order to increase
the accuracy of our predictions, we needed to verify when, where, and in
which direction trains were going. In our previous post (link to
Chloe’s), Chloe Mawer implemented a proof-of-concept Caltrain detector
using a webcam to acquire a video at our Mountain View offices. She
explained the use of OpenCV’s python bindings to walk through
frame-by-frame image processing. She showed that using video alone, it
is possible to positively identify a train based on motion from frame to
frame. She also showed how to use regions of interest within the frame
to determine the direction in which the Caltrain was traveling.

<center><video width="960" height="360" align="center" controls>
  <source src="video/confusion_matrix_movie.mp4" type="video/mp4">
</video></center>

The previous work was done using pre-recorded, hand-selected video.
Since our goal is to provide real time Caltrain detection, we had to
implement a streaming train detection algorithm and measure its
performance under real-world conditions. Thinking about a Caltrain
detector IoT device as a product, we also needed to slim down from a
camera + laptop to something with a smaller form factor. We already had
some experience [*listening to
trains*](http://www.svds.com/listening-caltrain/) using a Raspberry Pi,
so we bought a [*camera
module*](https://www.raspberrypi.org/products/pi-noir-camera/) for it
and integrated our video acquisition and processing/detection pipeline
onto one device. 

Caltrain detection, at its simplest, boils down to a simple question of
binary classification: Is there a train passing right now? Yes or no

<img src="figures/Binary_confusion_matrix.png" alt="OpenCV" width="150" align='right'>
As with any other binary classifier, the performance is defined by
evaluating the number of examples in each of four cases:
1. **Classifier says there is a train and there is a train, True Positive**
2. **Classifier says there is a train when there is none, False Positive**
3. **Classifier says there is no train when there is one, False Negative**
4. **Classifier says there is no train when there isn’t one, True Negative** 

For more info check out the blogs by Tom Fawcett, principal data scientist at
SVDS, on [classifier evaluation](http://www.svds.com/the-basics-of-classifier-evaluation-part-1)

After running our minimum viable Caltrain detector for a week, we began
to understand how our classifier performed, and importantly, where it
failed.

Causes of false positives:

-   Delivery trucks
-   Garbage trucks
-   Light rail
-   Freight trains

Causes of false negatives:

-   Darkness
-   Rain

## PiCamera and Video_Camera Class

Before diving into the data and how we solved these problems, let’s talk
about some of the nuts and bolts. How do we capture video and process it
on the Raspberry Pi?

The [*PiCamera*](https://picamera.readthedocs.io/) package is an
open-source package that offers a pure Python interface to the Pi camera
module that allows you to record image or video to file or stream. After
some experimentation, we decided to use PiCamera in a [continuous capture
mode](http://picamera.readthedocs.io/en/release-1.10/api_camera.html), as shown below in the **initialize_camera** and **initialize_video_stream** functions. 

The stream of still image frame captures are output as a numpy array representation of the image into a deque, a [double-ended queue](https://en.wikipedia.org/wiki/Double-ended_queue), for future processing. We decided to use a deque because we will need to add/remove/access objects from both the front (head) and back(tail) of the deque. Moreover, we can easily constrain the maximum length of our **input_deque** with the maxlen argument. 

<img src="figures/camera_codeblock_1_folded.png" alt="Smooth images" width="800" align='left'>

## Threading and task management in python

As you may have noticed, we implement our video_camera class as a new [thread](https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/4_Threads.html) using the python [threading](https://docs.python.org/2/library/threading.html) module. In order to perform real time train detection on a raspberry pi, threading is critical to ensure robust performance and minimization of data loss in our asynchronous detection pipeline. This is because multiple threads within a process (our python script) share the same data space with the main thread, facilitating:

-   Communication of information between threads

-   Interruption of individual threads without termination 

-   Most importantly, individual threads can be put to sleep (held in place) while other threads are running. This allows for nonparallel tasks to run without interruption on a single processor. 

<img src="figures/threading_diagram.png" alt="OpenCV" width="200" align='right'>
For example, imagine you are reading a book but are interrupted by a freight train rolling by your office. How would you be able to come back and continue reading from the exact place where you stopped? 

One way you could do this is by recording the page, line and word number. This way your execution context for reading a book are these 3 numbers! Now if your coworker is using the same technique, she can borrow the book and continue reading where she stopped before. When she is done, you can even take the book back and continue from where you were. Similar to reading a book with multiple people, or asynchronously processing video and audio signals, many tasks can share the same processor on the Raspberry Pi!

## Video_Sensor Class

Now that we are collecting and storing data from the PiCamera in the **input_deque**, we can create a new thread, the **video_sensor**, which asynchronously process these images independent of the video_camera thread. The job of the **video_sensor** is to determine which pixels have changed values overtime, i.e. motion. To do this, we will need to identify the background of the image, the non-moving objects in the frame which inadvertently mask motion, and the foreground of the image the new/moving objects in the frame. After we have identified motion, we will apply a 5x5 pixel kernal filter to reduce noise in our motion measurement via the [cv2.morphologyEx](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html) function. 

<img src="figures/camera_codeblock_2.png" alt="Smooth images" width="500" align='left'>

## Real time background subtraction masks

Chloe demonstrated(link) that we could detect trains with processed video feeds that isolate motion, through a process called background subtraction, by setting thresholds for the minimum intensity and duration of motion. Since background subtraction must be applied to each frame and the Pi has only modest computational speed, we needed to streamline the algorithm to reduce computational overhead. Luckily, [OpenCV 3](http://opencv.org/opencv-3-0.html) comes with multiple [background subtraction algorithms](http://docs.opencv.org/3.1.0/db/d5c/tutorial_py_bg_subtraction.html#gsc.tab=0) that run optimized C code with convenient Python APIs including:

-   [backgroundsubtractorMOG2](http://www.sciencedirect.com/science/article/pii/S0167865505003521) : A Gaussian Mixture-based Background/Foreground Segmentation Algorithm developed by Zivkovic and colleagues.  It uses a method to model each background pixel by an optimized mixture of K Gaussian distributions. The weights of the mixture represent the time proportions that those colours stay in the scene. The probable background colours are the ones which stay longer and more static.

<img src="figures/knn_theory.png" alt="OpenCV" width="200" align='right'>


-   [backgrounsubtractorKNN](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_knn/py_knn_understanding/py_knn_understanding.html) : KNN involves searching for the closest match of the test data in the feature space of historical image data. An example of this to the right, where we try and discern which class (blue square or red triangle) the new data (green circle) belongs to by factoring in not only the closest neighbor (red triangle), but the proximity threshold of k-nearest neighbors. For instance, if k=5 then the green circle would be assigned to the blue square class.  

We tested each and found that backgroundsubtractorKNN gave the best balance between rapid response to change and adaptability due to changing conditions, such as a distant tree fluttering in the wind. Moreover, the KNN method  can be improved through machine learning, and the classifer can be saved to file for repeated use. The cons of KNN include artifacts from full field motion, limited tutorials and documentation and that backgroundsubtractorKNN only runs on OpenCV 3.0 and higher.

<img src="figures/camera_codeblock_3.png" alt="Smooth images" width="600" align='left'>

<center><video width="960" height="360" align="center" controls>
  <source src="video/mog2_knn_comparison.mp4" type="video/mp4">
</video></center>

### Dynamically update camera settings in response to varied lighting

Discuss problems of false negatives to do decreasing light in the evening and at night. Look below at this graph. We need to be able to dynamically change the camera properties outside of its given settings. We also picked a camera without an IR filter to gather more light in the >700nm range which is more present at night (this is why some of our pictures seem discoloured compared to traditional cameras like your smart phone that does have an IR filter)

Discuss camera exposure problems
- Using the KNN background subtractor, sitll have lack of light at sunrise/sunset...

<img src="figures/sunrise_sunset_data.jpg" alt="Smooth images" width="800" align='left'>

Describe how we determine when to vary the camera settings. 
-    We record the intensity mean of the image, which the camera tries to keep around 128 at all times (half of the 8 bit 255 limit). 
-    Set up and lower limits of when to change the mask, we picked 1/8 intensity for night switch, when we empirically observed an inablity to reliably detect motion ~1 hour post sunset. And the high limit of post sunrise brightness requiring resetting back to day settings
-    Reset the mask to make sure that we do not falsely trigger train detection from the giant motion artificats that we create from changing all of the pixel values
-    Wait 1 second between setting camera settings and triggering the mask, to avoid thread lagging time before the PiCamera 2 changes it actions
-    Also do not continually trigger the night settings if it is already night, with checking if the operating mode is already what the camera would switch into
-    We also make sure that if the camera operating mode is already night, we do not take 

In [15]:
# Discuss solution of camera exposure problems with code block below

<img src="figures/camera_codeblock_4.png" alt="Smooth images" width="800" align='left'>

### Video_Detector Class

Talk about the different steps necessary to set up the detector. 

First Create_ROIS, similiar to Chloe, but have a center ROI to better detect train direction and speed with 3 data points instead of 2
Create buffers including
-    the motion detected buffer that has: length of motion_detected_buffer determines how time of motion translates into detection
- Train detection buffer which is the length of the cooldown period so a train can only be triggered once in a set amount of time
- Frame sampling buffers to keep a short term record of raw and processed frames for future analysis, plotting or saving

<img src="figures/video_detector_and_buffers_codeblock.png" alt="Smooth images" width="800" align='left'>

Talk about ROI to process, which is a row of information including time, motion from 3 ROIs, if motion was detected, if train was detected and the direction the train classifier declares
-    extract ROI to process and get average motion per ROI
-    check if motion exceeds the defined threshold and update buffers,roi_data
-    check if temporal threshold exceeded for detection (& cooldown not violated)
-    update the history dataframe and adjust the frame number pointer
-    add frames and prcessed motion and detection data to sampler

<img src="figures/roi_to_process_code.png" alt="Smooth images" width="800" align='left'>

### Persist proceesed data to Pandas Dataframe

Discuss need for persisting data
Briefly discuss why pandas dataframe chosen (easy for import, analysis and export of data: 
    pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. flexible open source data analysis / manipulation tool 

pandas is well suited for many different kinds of data:

Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
Ordered and unordered (not necessarily fixed-frequency) time series data.
Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure

In [ ]:
# Discuss the history dataframe class

<img src="figures/history_codeblock_5.png" alt="Smooth images" width="650" align='left'>

In [ ]:
Analysis of dataframe data

<img src="figures/history_dataframe_example.png" alt="Smooth images" width="600" align='left'>

### Detector_Worker Class

Want to be able to visualize this history dataframe as well as raw and unprocessed data. This requires additional processing time and resources, and we do not want to interrupt the video detector, so therefore create the detector worker class.



<img src="figures/Frame_Sampler_Example_Pics_Only.png" alt="Smooth images" width="650" align='left'>

Video plotter and frame sampler module use code that extracts out information from the history dataframe when instructed and then can be used 

### Train Direction

<img src="figures/Direction_Detection.png" alt="Smooth images" width="650" align='left'>

We all tried to design different methods, and they had varying results

-    Static ‘Boolean’ Method (Chloe) - Track activity of motion thresholds seperately in ROIs, select N/S depending on which train passed first. Does not work well for express trains which trigger in motion simultaneously


-    Streaming ‘Integration’ Method (Colin) - Sum the value of the motion, but is sensitive to changes in camera position or not accurate declaration of ROI location

<img src="figures/Direction_Detection_Code_Small.png" alt="OpenCV" width="300" align='right'>
-    Streaming ‘Curve-Fit’ Method (Matt R) - Determine empirically fitting to a sigmoid curve where the ROI sensor hits 50% of the max value. If it never exceeds half of the max value, revert back to Chloe's static boolean method. This also allows determination of train speed if the real distance between the ROIs is known. 

<img src="figures/Direction_Detection_Code_Big.png" alt="Smooth images" width="650" align='left'>

Determination of train speed will be covered in *Streaming Audio Analysis and IoT Sensor Fusion*. Importantly other false positives like light rails or large trucks that pass in front of the camera also trigger the sensor. By having a secondary data feed, i.e. audio, we can have a second input to determine if a train is passing by both visual and sound cues. Later in the trainspotting series we will also cover how to reduce false positive of freight trains using image recognition via *TensorFlow and Neural Nets for Recognizing Images on a Raspberry Pi*

<img src="figures/Pi_Video_Only_Architecture.png" alt="Smooth images" width="1200" align='left'>

We hope that you now understand how to design your own architecture for stream video processing on an IOT device. On our Raspberry Pi 3B, our pipeline consists of hardware and software running on top of [Raspbian Jesse](https://www.raspberrypi.org/blog/raspbian-jessie-is-here/), a derivative of Debian Linux. Highlighted in green are our 3 major components for acquiring, processing, and evaluating streaming video:
-   **Video Camera**: Initializes picamera and captures frames from the video stream
-   **Video Sensor:** Processes the captured frames and dynamically varies video camera settings 
-   **Video Detector**: Determines motion in specifed aReas of Interst (ROIs), and evaluates if a train passed

In addition to our main camera, sensor and detector processes, several sub-classes (orange) are needed to perform image background subtraction, persist data, and run models:
-   **Mask**: Performs background subtraction on raw images, using powerful algorithims implented in OpenCV 3.0 
-   **History**: An accessible [Pandas](http://pandas.pydata.org) dataframe that is updated in real time to persist data and faciliates SQL-like queries
-   **Detector Worker**: Assists the video detector in evaluating image, motion and history data. This class consists of several modules (yellow) responsible for sampling frames from the video feed, plotting data and running models to determine train direction.

All of the software is written in [python 2.7](https://www.python.org) and can be controlled from a [Jupyter Notebook](http://jupyter.org) run locally on the Pi or remotely on your laptop. Trainspotting blog posts including *Connecting an IoT device to the Cloud* and *How to Build a Deployable IoT Device using a Raspberry Pi* will cover using a remote server to control a Raspberry Pi in greater detail! 